In [1]:
using BenchmarkTools: @btime
import LinearAlgebra as la

In [2]:
struct HO_Funcs
    ω::Float64
    hermites::Vector{Float64}
    hos::Vector{Float64}
    ho_dder::Vector{Float64}
    
    function HO_Funcs(l, ω)
        hermites = zeros(l)
        hermites[1] = 1.0
        
        hos = zeros(l)
        ho_dder = zeros(l)
        return new(ω, hermites, hos, ho_dder)
    end
end

In [11]:
function fast_ho_dder(x, ho)
    ω, H_ξ, ψ_HO, ψ_HO_dder = ho.ω, ho.hermites, ho.hos, ho.ho_dder
    
    ψ_HO_dder[1] = ω * (ω * x^2 - 1) * ψ_HO[1]
    ψ_HO_dder[2] = ω * ((ω * x^2 - 1) * ψ_HO[2] - √ω * x * 4 * ψ_HO[1])

    for n in 2:length(ψ_HO)-1
        ψ_HO_dder[n+1] = ω * ((ω * x^2 - 1) * ψ_HO[n+1] - √ω * x * 4n * ψ_HO[n] + 4 * (n-1)*n * ψ_HO[n - 1])
    end
    
    return ψ_HO_dder
end

fast_ho_dder (generic function with 1 method)

In [16]:
@btime ψ_HO_dder = fast_ho_dder(x, ho);

  101.810 ns (0 allocations: 0 bytes)


In [24]:
function fast_ho(x, ho)
    ω, H_ξ, ψ_HO, ψ_HO_dder = ho.ω, ho.hermites, ho.hos, ho.ho_dder
    
    ξ = √ω * x
    H_ξ[1] = 1
    H_ξ[2] = 2ξ
    
    ho_fac = (ω / π)^0.25 * exp(-ξ^2 / 2)
    ψ_HO[1] = ho_fac * H_ξ[1]
    #ψ_HO_dder[1] = ho_fac * ω * ((ω * x^2 - 1) * H_ξ[1])
    
    ho_fac *= 1 / √2
    ψ_HO[2] = ho_fac * H_ξ[2]
    #ψ_HO_dder[2] = ho_fac * ω * ((ω * x^2 - 1) * H_ξ[2] - √ω * x * 4 * H_ξ[1])

    for n in 2:length(ψ_HO)-1
        H_ξ[n+1] = 2ξ * H_ξ[n] - 2(n-1) * H_ξ[n-1]

        ho_fac *= 1 / sqrt( 2n )
        ψ_HO[n+1] = ho_fac * H_ξ[n+1]
        #ψ_HO_dder[n+1] = ho_fac * ω * ((ω * x^2 - 1) * H_ξ[n+1] - √ω * x * 4n * H_ξ[n] + 4 * (n-1)*n * H_ξ[n - 1])
    end
    
    return ψ_HO, ψ_HO_dder
end

fast_ho (generic function with 1 method)

In [4]:
l = 50
ω = 0.25
ho = HO_Funcs(l, ω);

In [5]:
x = 8.1
ψ_HO, ψ_HO_dder = fast_ho(x, ho);

In [25]:
@btime ψ_HO, ψ_HO_dder = fast_ho(x, ho);

  193.667 ns (1 allocation: 32 bytes)


In [241]:
function ho_test(x)
    n = 5
    x = √ω * x
    hermite = 32x^5 - 160x^3 + 120x
    return (ω/π)^0.25 * 1 / sqrt(2^n * factorial(n)) * hermite * exp(-x^2/2)
end

ho_test (generic function with 1 method)

In [242]:
import ForwardDiff as fd

In [243]:
println(ho_test(8.1))
println(ψ_HO[6])

0.0581314323587515
0.05813143235875152


In [244]:
fd.derivative(x -> fd.derivative(ho_test, x), 8.1)

0.07851376582953867

In [245]:
ψ_HO_dder[6]

0.07851376582953876

In [246]:
ψ_HO_dder

50-element Vector{Float64}:
  0.0005610106947175589
  0.002795990940600953
  0.00934029097666905
  0.023867492077149124
  0.04856101035344266
  0.07851376582953876
  0.09524692095641227
  0.06612135851419047
 -0.04139724039733315
 -0.22813140693393136
 -0.42915580566302036
 -0.5110467066417964
 -0.3327439087257611
  ⋮
 -2.905488127663094
 -0.48138866465950403
  2.6084385730990287
  2.911340684868396
 -0.08652195993288064
 -3.127880932336111
 -2.686801388131804
  0.9109446885618342
  3.5937592397223894
  2.131279871030662
 -1.935062436122154
 -3.8393761225775864